In [4]:
%load_ext autoreload
%autoreload 2

In [17]:
import pandas as pd
import bindome as bd
from os.path import join
import os
import anndata

## Load h5ad file (if already processed)

In [18]:
bd.constants.ANNOTATIONS_DIRECTORY = '../annotations'

In [19]:
ad = bd.datasets.PBM.uniprobe()

/home/rio/miniconda3/envs/multibind/lib/python3.10/site-packages/anndata/_core/anndata.py:1828: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


### Prepare h5ad file (if not exists)

In [20]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
uniprobe = os.path.join(bd.constants.ANNOTATIONS_DIRECTORY, 'uniprobe', 'All_deBruijn')
ad = []
for f1 in os.listdir(uniprobe):
    df = []
    if f1.endswith('.zip'):
        continue
    print(f1)
    next_dir = join(uniprobe, f1)
    for f2 in os.listdir(next_dir):
        p = join(next_dir, f2)
        print(p)
        df2 = pd.read_csv(p, sep='\t', header=None)
        # print(p, df.shape)
        df2.columns = ['intensity', 'sequence'] if 'A' in df2[1].values[0] else ['intensity', 'sequence'][::-1]
        df2 = df2[~df2['sequence'].str.contains('Sequence')]
        df2['filename'] = f2.replace('.txt', '')
        df2['intensity'] = df2['intensity'].astype(str).str.len() > 0
        df2['intensity'] = df2['intensity'].astype(float)
        df2['dataset.id'] = f1
        
        # remove duplicates
        df2['k'] = df2['dataset.id'] + ':' + df2['filename']
        df2['k2'] = (df2['k'] + ':' + df2['sequence'])
        df2 = df2.drop_duplicates('k2')
        # print(df2.shape)
        df.append(df2)
        # if len(df) >= 1:
        #     break
    
    df = pd.concat(df).reset_index(drop=True)
    
    ad2 = anndata.AnnData(df.pivot('filename', 'sequence', 'intensity'))
    ad2.obs['dataset.id'] = f1
    ad.append(ad2)

ad = anndata.concat(ad, join='outer')

print(ad.shape)
ad.write(join(bd.constants.ANNOTATIONS_DIRECTORY, 'uniprobe', 'All_deBruijn.h5ad'), compression='lzf')

CB11
../annotations/uniprobe/All_deBruijn/CB11/LUX_015681_rep1_combinatorial.txt
../annotations/uniprobe/All_deBruijn/CB11/LUX_015681_rep2_combinatorial.txt
../annotations/uniprobe/All_deBruijn/CB11/LUX_015681_rep3_combinatorial.txt
../annotations/uniprobe/All_deBruijn/CB11/LUX_016060_rep1_combinatorial.txt
../annotations/uniprobe/All_deBruijn/CB11/LUX_016060_rep2_combinatorial.txt
../annotations/uniprobe/All_deBruijn/CB11/LUX_016060_rep3_combinatorial.txt
Cell08
../annotations/uniprobe/All_deBruijn/Cell08/Alx3_3418.2_deBruijn.txt
../annotations/uniprobe/All_deBruijn/Cell08/Alx4_1744.1_deBruijn.txt
../annotations/uniprobe/All_deBruijn/Cell08/Arx_1738.2_deBruijn.txt
../annotations/uniprobe/All_deBruijn/Cell08/Bapx1_2343.1_deBruijn.txt
../annotations/uniprobe/All_deBruijn/Cell08/Barhl1_2590.2_deBruijn.txt
../annotations/uniprobe/All_deBruijn/Cell08/Barhl2_3868.1_deBruijn.txt
../annotations/uniprobe/All_deBruijn/Cell08/Barx1_2877.1_deBruijn.txt
../annotations/uniprobe/All_deBruijn/Cell08/